In [3]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-141819")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141819
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-141819


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "project1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    #(workspace=ws, name=compute_cluster_name, provisioning_configuration=compute_cluster_config)
    compute_target.wait_for_completion(show_output=True)
        
print(compute_target.get_status().serialize())

Found existing compute target
{'errors': [], 'creationTime': '2021-04-02T15:11:50.195338+00:00', 'createdBy': {'userObjectId': 'dd4af0a4-a6c4-481c-bff0-d545cc1d27b7', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 141819'}, 'modifiedTime': '2021-04-02T15:14:21.398892+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS12_V2'}


In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice , normal
import os

# Specify parameter sampler
parameter_sampler = RandomParameterSampling( {
        "--C": uniform(0.05, 0.1),
        "--max_iter": choice(16, 32, 64, 128)}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5) ### YOUR CODE HERE ###
"""Bandit terminates runs where the primary metric is not within 
the specified slack factor/slack amount compared to the best performing run."""

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", 
    compute_target=compute_target, entry_script="train.py")### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = parameter_sampler,
                                     policy = policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs = 5
                                     )### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [16]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show() 
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Accuracy: ', best_run_metrics['Accuracy'])

best_run.get_file_names() #To get the actual model file
### YOUR CODE HERE ###

Best Run Id:  HD_82a26817-1a5f-48e9-a196-9fdc16830faa_1
Accuracy:  0.9111785533636824


['azureml-logs/55_azureml-execution-tvmps_475fe729d466a93d4ec7ad9d1df11ae5a11d13d70d63d5033b3f2cba5cb00c7c_d.txt',
 'azureml-logs/65_job_prep-tvmps_475fe729d466a93d4ec7ad9d1df11ae5a11d13d70d63d5033b3f2cba5cb00c7c_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_475fe729d466a93d4ec7ad9d1df11ae5a11d13d70d63d5033b3f2cba5cb00c7c_d.txt',
 'logs/azureml/102_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [18]:
best_run.download_file(name="outputs/model.joblib", output_file_path="./outputs/")

In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_82a26817-1a5f-48e9-a196-9fdc16830faa_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

Url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=Url)

### YOUR CODE HERE ###

In [22]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
    
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state= 150)
    
data = pd.concat([x_train,y_train],axis=1)
data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
2583,34,1,0,0,1,4,1,96,1,999,...,0,0,0,1,0,0,0,0,0,0
15456,57,1,0,1,0,8,1,154,2,999,...,0,0,0,1,0,0,0,0,0,0
20498,39,0,0,0,0,5,3,64,1,999,...,1,1,0,0,0,0,0,0,0,0
9424,31,1,0,1,0,6,4,130,2,999,...,1,0,0,0,0,0,0,1,0,0
6671,28,0,0,0,0,7,3,96,2,999,...,0,0,0,0,0,0,0,1,0,0


In [23]:

if not os.path.isdir('data'):
    os.mkdir('data')

pd.DataFrame(data).to_csv("data/train_data.csv", index=False)   #saving the train data to a csv file

ds1 = ws.get_default_datastore()
ds1.upload(src_dir='./data', target_path='solution', overwrite=True, show_progress=True)

train_data = TabularDatasetFactory.from_delimited_files(path = ds1.path("solution/train_data.csv"))
label = 'result'

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [31]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= "accuracy",
    #compute_target=compute_target,
    training_data= data,
    label_column_name= 'y',
    n_cross_validations= 4)

In [32]:
run_auto_ml = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_6b166597-1779-4d57-9840-0f0e2459daab

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [33]:
# Printing the Run details 
RunDetails(run_auto_ml).show()
# Ensuring that AutoML run is completed
run_auto_ml.wait_for_completion(show_output=True)
### YOUR CODE HERE ###"""

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|2971                             |1                                |26360                                 |
+---------------------------------+----------

{'runId': 'AutoML_6b166597-1779-4d57-9840-0f0e2459daab',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-04-02T16:23:00.423482Z',
 'endTimeUtc': '2021-04-02T16:58:33.494731Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-141819","workspace_name":"quick-starts-ws-141819","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y_max":null,"n

In [34]:
# Retrieve and save your best automl model.

best_run, best_automl_model = run_auto_ml.get_output()
print(best_run)
print(best_automl_model)

import joblib
joblib.dump(best_automl_model, "outputs/best_automl_model.joblib")
best_run.register_model("auto-ml" , model_path = './outputs/')
### YOUR CODE HERE ###

Run(Experiment: AutoML,
Id: AutoML_6b166597-1779-4d57-9840-0f0e2459daab_34,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                max_iter=100,
                                                                                                multi_class='ovr',
                                                            

Model(workspace=Workspace.create(name='quick-starts-ws-141819', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-141819'), name=auto-ml, id=auto-ml:1, version=1, tags={}, properties={})

In [35]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/15540e04/classes.interpret.json',
 'explanation/15540e04/eval_data_viz.interpret.json',
 'explanation/15540e04/expected_values.interpret.json',
 'explanation/15540e04/features.interpret.json',
 'explanation/15540e04/global_names/0.interpret.json',
 'explanation/15540e04/global_rank/0.interpret.json',
 'explanation/15540e04/global_values/0.interpret.json',
 'explanation/15540e04/local_importance_values.interpret.json',
 'explanation/15540e04/per_class_names/0.interpret.json',
 'explanation/15540e04/per_class_rank/0.interpret.json',
 'explanation/15540e04/per_class_values/0.interpret.json',
 'explanation/15540e04/rich_metadata.interpret.json',
 'explanation/15540e04/true_ys_viz.interpret.json',
 'explanation/15540e04/visualization_dict.interpret.json',
 'explanation/c02560e3/classes.interpret.json',
 'explanation/c02560e3/expected_values.interpret.json',
 'explanation/c02560e3/features.interpret.json',
 'explanation/c02560e3/global_names/0.interpret.json',
 'explanation/c02560e3/global_rank/0.interpret.json',
 'explanation/c02560e3/global_values/0.interpret.json',
 'explanation/c02560e3/local_importance_values.interpret.json',
 'explanation/c02560e3/per_class_names/0.interpret.json',
 'explanation/c02560e3/per_class_rank/0.interpret.json',
 'explanation/c02560e3/per_class_values/0.interpret.json',
 'explanation/c02560e3/rich_metadata.interpret.json',
 'explanation/c02560e3/true_ys_viz.interpret.json',
 'explanation/c02560e3/visualization_dict.interpret.json',
 'explanation/c02560e3/ys_pred_proba_viz.interpret.json',
 'explanation/c02560e3/ys_pred_viz.interpret.json',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [36]:
best_run.get_metrics()

{'balanced_accuracy': 0.7631867317629587,
 'f1_score_macro': 0.7787003425827954,
 'average_precision_score_micro': 0.9812588932807949,
 'average_precision_score_weighted': 0.9553732719919897,
 'weighted_accuracy': 0.9549825168319106,
 'average_precision_score_macro': 0.8256191625921945,
 'recall_score_micro': 0.916578148710167,
 'AUC_weighted': 0.9470405330740066,
 'norm_macro_recall': 0.5263734635259173,
 'precision_score_micro': 0.916578148710167,
 'f1_score_weighted': 0.9139942575669713,
 'recall_score_macro': 0.7631867317629587,
 'AUC_macro': 0.9470405330740066,
 'accuracy': 0.916578148710167,
 'log_loss': 0.31840312728962433,
 'precision_score_macro': 0.7975725667264932,
 'AUC_micro': 0.9804559662983183,
 'recall_score_weighted': 0.916578148710167,
 'matthews_correlation': 0.5595800843447807,
 'f1_score_micro': 0.916578148710167,
 'precision_score_weighted': 0.9122975448375937,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_6b166597-1779-4d57-9840-0f0e2459daab_34

In [ ]:
compute_target.delete()